In [1]:
using JuMP
using CPLEX
using LinearAlgebra


Now we define the master problem :)

In [26]:
function solve_master(A,b,c,h,T,V,R)
    println("Beginning Master ...")
    relaxation = 15
    nx = size(c,1)
    m = Model(CPLEX.Optimizer)
    @variable(m, x[1:nx] >= 0)
    @variable(m, theta)
    if(size(V,1) > 0)
        @objective(m, Min, sum(c[i]*x[i] for i = 1:nx) + theta)
    else
        @objective(m, Min, sum(c[i]*x[i] for i = 1:nx))
    end
    @constraint(m, A*x .<= b)
    if size(R,1) > 0
        for i in 1:size(R,1)
            @constraint(m, transpose(R[i,:])*(h[:,i]-T*x)<=0)
        end
    end
    if size(V,1) > 0
        for i in 1:size(V,1)
            println(h)
            println(T)
            println(x)
            println(T*x)
            @constraint(m, theta >= transpose(V[i,:])*(h-T*x))
        end
    end
    optimize!(m)
    sol = zeros(1, nx)
    for i = 1:nx
        sol[i] = JuMP.value(x[i])
    end
    sol = vec(sol)
    if(size(V,1)>0)
        return (objective_value(m), sol, theta, JuMP.termination_status(m))
    else
        return (objective_value(m), sol, -Inf, JuMP.termination_status(m))
    end
    println("Master Fini !")
end

solve_master (generic function with 1 method)

In [27]:
function step2(W, h, T, x, omega, q)
    println("Beginning step 2 ...")
    m = Model(CPLEX.Optimizer)
    ny = size(W,2)
    
    @variable(m, y[1:ny] >= 0)

    @constraint(m,constraint, W*y - h[:,omega] + T*x .<= 0)
    @objective(m, Min, transpose(q[:,omega]) * y)
    optimize!(m);
    sol = zeros(1, ny)
    for i = 1:ny
        sol[i] = JuMP.value(y[i])
    end
    sol = vec(sol)
    return (objective_value(m), sol, dual.(constraint), JuMP.termination_status(m))
    println("Step 2 Fini :)")
end

step2 (generic function with 1 method)

In [28]:
function main(A, b, c, T, W, h, q, p)
    V = []
    S = []
    
    k = 0
    val_k, x_k, theta_k = 0, 0, 0

    while (k<10000000000)
        println(k)
        
        (val_k, x_k, theta_k, termination_status) = solve_master(A, b, c, h, T, V, S)
    
        if termination_status == MOI.INFEASIBLE
            return (-1, -1, -1)
        end
        
        V_temp = []
        status = 0
        
        for w in 1:size(p,2)
            (pi_k, y_k, sigma_k, termination_status) = step2(W, h, T, x_k, w, q)
            
            if termination_status == MOI.INFEASIBLE
                S_k = [S_k; sigma_k]
                k += 1
                status = 1
                break
            else
                if V_temp == []
                    V_temp = [p[w]*pi_k]
                else
                    V_temp = [V_temp p[w]*pi_k]
                end
            end
        end
        if(status == 0)
            if V == []
                V_new  = V_temp
            else
                V_new = [V; V_temp]
            end
            
            if(V_new == V)
                return (x_k, y_k, k)
            else
                k += 1
                V = V_new
            end
        end
    end
end
        
            
            
        
        
    
        
    
    

main (generic function with 1 method)

In [29]:
d = [500 100; 300 300]
q = [-24 -28; -28 -32]
p = [0.4 0.6]

A = [1 1; -1 0; 0 -1]
b = [120; -40; -20]
c = [100; 150]

T = [-60 0; 0 -80; 0 0; 0 0]
W = [6 10; 8 5; 1 0; 0 1]
h = [0 0; 0 0; d[1,1] d[2,1]; d[1,2] d[2,2]]

(x,y, k) = main(A, b, c, T, W, h, q, p)

println(x)
println(y)
println(k)


0
Beginning Master ...
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
Tried aggregator 1 time.
LP Presolve eliminated 3 rows and 3 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.00 ticks)
Beginning step 2 ...
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
Tried aggregator 1 time.
LP Presolve eliminated 4 rows and 2 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.00 ticks)
Beginning step 2 ...
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
Tried aggregator 1 time.
LP Presolve eliminated 2 rows and 0 columns.
Reduced LP has 2 rows, 2 columns, and 4 nonzeros.
Presolve time = 0.00 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =        -10320.000000
1
Beginning Master ...
[0 0; 0 0; 500 300; 100 300]
[-60 0; 0 -80; 0 0; 0 0]
VariableRef[x[1], x[2]]
AffExpr[-60 x[1], -80 x[2], 0, 0]


LoadError: `promote_operation(-, Matrix{Int64}, Vector{AffExpr})` not implemented yet, please report this.